In [ ]:
import warnings
import torch
import pandas as pd
import numpy as np
import scanpy as sc
import os
import random
from torch.backends import cudnn
import episcanpy.api as epi
from PRESENT import PRESENT_function

warnings.filterwarnings("ignore")
sc.set_figure_params(dpi=80, figsize=(4,4), facecolor="white")

In [ ]:
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.enabled=False
    cudnn.benchmark = False
    torch.use_deterministic_algorithms(True,warn_only=True)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' 

In [ ]:
set_seed(42)

In [ ]:
adata_atac = sc.read_h5ad("./data/MISAR_seq/E15_5-S1_atac_gt.h5ad")
print(adata_atac)
adata_rna = sc.read_h5ad("./data/MISAR_seq/E15_5-S1_expr_gt.h5ad")
print(adata_rna)

In [ ]:
adata = PRESENT_function(
    spatial_key = "spatial", ## obsm key under which to load the spatial matrix of spots
    adata_rna = adata_rna, ## The RNA raw fragment count matrix of spots in anndata.AnnData format
    gene_min_cells = 1, ## Minimum number of cells expressed required for a gene to pass filtering
    num_hvg = 3000, ## Number of highly variable genes to select for RNA data
    adata_atac = adata_atac, ## The ATAC raw fragment count matrix of spots in anndata.AnnData format
    peak_min_cells_fraction = 0.03, ## Minimum fraction of cells accessible required for a peak to pass filtering
    nclusters = 12,
    device = "cuda" ## Device used for training: cuda or cpu
)
print(adata)

In [ ]:
sc.pl.embedding(adata, basis='spatial', color='LeidenClusters', title='PRESENT', s=90, show=False)

